# Uczenie ze wzmocnieniem
W tym ćwiczeniu przujrzymy się prostej implementacji uczenia ze wzmocnieniem z wykorzystaniem równań Belmana dla funkcji $Q$.

Importujemy potrzebne biblioteki:

In [24]:
import numpy as np
import matplotlib.pylab as plt
from copy import deepcopy
import random

## Przygotowania: tworzymy środowisko
Zdefiniujmy przydatne symbole za pomocą tórych będziemy rysować sytuacje:

In [25]:
S = "X" # ściana
P = " " # pusty korytarz
K = "#" # kara
N = "$" # nagroda

R = "@" # robot

Za ich pomocą tworzymy siatkę, która będzie stanowiła świat, w którym żyje sobie robot:

In [26]:
siatka = [[S,S,S,S,S,S],
        [S,P,P,P,N,S],
        [S,P,S,P,K,S],
        [S,P,P,P,P,S],
        [S,S,S,S,S,S]        
       ]
N_wierszy = len(siatka)
N_kolumn = len(siatka[0])

Umieśćmy w tym świecie robota:

In [27]:
robot_pos=[3,4] # wiersz,kolumna

Do wizualizacji sytuacji w jakiej aktualnie jest robot pryda nam się następująca funkcja:

In [28]:
def sytuacja(siatka, robot_pos):
    siatka_z_robotem = deepcopy(siatka)
    siatka_z_robotem[robot_pos[0]][robot_pos[1]] = R
    print(12*"_")
    for w in siatka_z_robotem:
        print(' '.join(w))
    print(12*"_")

A zatem aktualna sytuacje wygląda tak:

In [29]:
sytuacja(siatka, robot_pos)

____________
X X X X X X
X       $ X
X   X   # X
X       @ X
X X X X X X
____________


Zdefiniujemy możliwe punkty startu robota (dowolne miejsce w pustym korytarzu):

In [30]:
punkty_startu = []
for w_i, w in enumerate(siatka):
    for k_i,k in enumerate(w):
        if k == P:
            punkty_startu.append([w_i,k_i])
print(punkty_startu)
print(len(punkty_startu))

[[1, 1], [1, 2], [1, 3], [2, 1], [2, 3], [3, 1], [3, 2], [3, 3], [3, 4]]
9


## Akcje 
Zdefiniujmy możliwe akcje:

In [31]:
UP = 0
DOWN = 1
LEFT = 2
RIGHT = 3

AKCJE = [UP, DOWN, LEFT, RIGHT]

Do wykonania wybranej akcji użyjemy poniższej funkcji. Sprawdza ona czy wybrana akcja prowdzi na nowe miejsce (nie na ścianę) i jeśli tak to zmienia pozycję robot. W przeciwnym razie robot pozostanie na poprzedniej pozycji.

In [32]:
def nowa_pozycja_robota(robot_pos, siatka, akcja):
    p = deepcopy(robot_pos)

    w = robot_pos[0] # wiersz w którym jest robot
    k = robot_pos[1] # kolumna w której jest robot
    
    if akcja == UP: # próbujemy zmniejszyć numer wiersza o 1 (chyba, że jest tam ściana)
        co_jest_wyzej = siatka[w - 1][k]
        if co_jest_wyzej != S: # jak nie ściana to idzie wyzej
            p[0] = w - 1
    elif akcja == DOWN:
        co_jest_nizej = siatka[w + 1][k]
        if co_jest_nizej != S: # jak nie ściana to idzie niżej
            p[0] = w + 1
    elif akcja == LEFT:
        co_jest_na_lewo = siatka[w][k -1]
        if co_jest_na_lewo != S: # jak nie ściana to idzie w lewo
            p[1] = k - 1
    elif akcja == RIGHT:
        co_jest_na_prawo = siatka[w][k + 1]
        if co_jest_na_prawo != S: # jak nie ściana to idzie w prawo
            p[1] = k + 1
    return p
        

Kolejna funkcja będzie wykonywać ruch i wypłacać nagrodę: 

In [33]:
def act(robot_pos, siatka, akcja):            
    p = nowa_pozycja_robota(robot_pos, siatka, akcja) # próba przejścia do nowej pozycji zgodnie z wybraną akcją
    obiekt_w_nowej_pozycji_robota = siatka[p[0]][p[1]] # sprawdzamy w co wdepnął robot
    
    if obiekt_w_nowej_pozycji_robota == K: # dostaje karę
        nagroda = -1000
        is_done = True
        pozycja_robota = p # przesuwam się na nowe miejsce
    elif obiekt_w_nowej_pozycji_robota == N: # dostaje nagrodę
        nagroda = 100
        is_done = True
        pozycja_robota = p # przesuwam się na nowe miejsce
    elif obiekt_w_nowej_pozycji_robota == P: # przesuwam się wzdłóż korytarza
        nagroda = -1
        is_done = False
        pozycja_robota = p # przesuwam się na nowe miejsce
    return p, nagroda, is_done

Poniższa funkcja zapewnia dynamikę naszemu robotowi. Zasadniczo powinien on postępować zgodnie z akcją odpowiadającą największej wartości w tabeli $Q$ reprezentowanej tu przez tablicę `q_table`. Jednak aby zapewnić mu możliwość eksploracji świata trzeba mu dać możliwość wyboru losowego kroku:

In [34]:
def choose_akcja(robot_pos):
    if random.uniform(0, 1) < eps:
        a = random.choice(AKCJE)  # losowa akcja
    else:
        a = np.argmax(q_table[robot_pos[0], robot_pos[1],:]) # akcja zgodna z aktualną strategią
    return a 

## Strategia:
Jeszcze jedna funkcja pomocnicza. Ilustruje ona aktualną strtegię wypisując strzałki pokazujące ruchy optymalne wq. tabeli $Q$:

In [35]:
def strategia(q_table):
    Q_max = np.zeros((N_wierszy,N_kolumn),dtype='unicode')
    Q_max[:] = ' '
    for w in range(1,N_wierszy-1):
        for k in range(1,N_kolumn-1):
            a = np.argmax(q_table[w,k])
            if  a==0: 
                Q_max[w][k] = '\u2191'
            elif a ==1:
                Q_max[w][k] = '\u2193'
            elif a ==2:
                Q_max[w][k] = '\u2190'
            elif a ==3:
                Q_max[w][k] = '\u2192'
    Q_max[2,2] =' '   
    Q_max[1,4] = ' '
    Q_max[2,4] = ' '

    print(12*"_")
    for w in Q_max:
        print(' '.join(w))
    print(12*"_")        

Zainicjujmy tabelę $Q$ wpisując wszędzie liczby losowe:

In [36]:
q_table = np.random.randint(0,4,(N_wierszy, N_kolumn, len(AKCJE)))

Tabela ta wygląda tak:

In [14]:
print(q_table)

[[[0 3 3 0]
  [3 1 3 1]
  [0 2 1 2]
  [1 1 0 2]
  [3 3 1 1]
  [1 1 3 3]]

 [[0 3 0 2]
  [3 1 0 3]
  [3 3 2 3]
  [3 1 2 1]
  [0 0 3 3]
  [1 0 1 0]]

 [[3 2 2 0]
  [0 2 3 3]
  [2 2 2 2]
  [0 2 1 3]
  [1 0 3 0]
  [0 2 1 2]]

 [[2 0 3 0]
  [2 2 2 2]
  [0 3 0 3]
  [2 2 1 2]
  [0 3 3 2]
  [0 0 2 3]]

 [[1 2 1 2]
  [3 1 0 2]
  [2 1 3 3]
  [2 0 2 2]
  [3 0 0 3]
  [2 1 3 1]]]


Wartości dla poszczególnych akcji w pozycji [3,4] są następujące:

In [38]:
sa = q_table[3,4]
print("up={}, down={}, left={}, right={}".format(sa[UP], sa[DOWN], sa[LEFT] , sa[RIGHT]))

up=2, down=2, left=3, right=2


Strategicznie jest wybrać akcję o największej wartości $Q$.

In [39]:
print('Optymalna akcja o indeksie:')
np.argmax(q_table[3,4])

Optymalna akcja o indeksie:


2

Cała strategia wygląda tak:

In [40]:
strategia(q_table)

____________
           
  ← ↑ ←    
  ←   ↓    
  ↑ → ↓ ←  
           
____________


## Symulacja i nauka
### Definiujemy stałe określające symulację:

In [41]:
N_EPISODES = len(punkty_startu)  #  z ilu punktów spróbujemy wystartować
MAX_EPISODE_STEPS = 60 # ile kroków max zrobimy z danego punktu startu
ILE_RAZY = 30 # ile razy powtórzymy start z każdego punktu

* Parametry określające szybkość uczenia. Prędkość uczenia będziemy stopniowo zmiejszać w czasie kolejnych przejść przez punkty startowe:

In [42]:
MIN_ALPHA = 0.02
alphas = np.linspace(1.0, MIN_ALPHA, ILE_RAZY) 

* Współczynnik propagacji błędów

In [43]:
gamma = 1 

 * poziom losowości akcji

In [44]:
eps = 0.2

### Nauka
Z losowych miejsc puszczamy robota. Prubuje on wykonywać akcje zgodne z aktualną strategią wynikającą z tablicy $Q$ (z dokładnością do założonej losowości). Za pomocą równania Belmana uaktualniana jest tablica $Q$. W czasie nauki postępy podglądamy obserwując ruchy robota w labiryncie. Po każdym przejściu przez wszystkie punkty wyświetlamy strategię:

In [45]:
for p in range(ILE_RAZY):
    random.shuffle(punkty_startu)
    alpha = alphas[p]
    for s in range(N_EPISODES):
        robot_pos=punkty_startu[s]
        total_nagroda = 0
        

        for _ in range(MAX_EPISODE_STEPS):
            akcja = choose_akcja(robot_pos)
            nowa_poz_robota, nagroda, done = act(robot_pos,siatka, akcja)
            total_nagroda += nagroda
            # Równanie Belmana dla Q:
            q_table[robot_pos[0],robot_pos[1], akcja] += \
                  alpha * (nagroda + gamma *  np.max(q_table[nowa_poz_robota[0],nowa_poz_robota[1],:] 
                                                   - q_table[robot_pos[0],robot_pos[1], akcja]))
            robot_pos = nowa_poz_robota
            sytuacja(siatka, robot_pos)

            if done:
                break
        print("Episod {}: total nagroda -> {}".format(p + 1, total_nagroda))
        print("Aktualna najlepsza strategia:")
        strategia(q_table)

____________
X X X X X X
X       $ X
X @ X   # X
X         X
X X X X X X
____________
____________
X X X X X X
X       $ X
X @ X   # X
X         X
X X X X X X
____________
____________
X X X X X X
X       $ X
X @ X   # X
X         X
X X X X X X
____________
____________
X X X X X X
X       $ X
X   X   # X
X @       X
X X X X X X
____________
____________
X X X X X X
X       $ X
X @ X   # X
X         X
X X X X X X
____________
____________
X X X X X X
X       $ X
X   X   # X
X @       X
X X X X X X
____________
____________
X X X X X X
X       $ X
X   X   # X
X @       X
X X X X X X
____________
____________
X X X X X X
X       $ X
X   X   # X
X @       X
X X X X X X
____________
____________
X X X X X X
X       $ X
X @ X   # X
X         X
X X X X X X
____________
____________
X X X X X X
X       $ X
X   X   # X
X @       X
X X X X X X
____________
____________
X X X X X X
X       $ X
X @ X   # X
X         X
X X X X X X
____________
____________
X X X X X X
X       $ X
X @ X   # X
X    

Ostateczna strategia:

In [46]:
strategia(q_table)

____________
           
  → → →    
  ↑   ↑    
  ↑ ← ↑ ←  
           
____________


## Rzeczy do wypróbowania:
* jak wpływają na naukę 
  * proporcje kary i nagrody
  * współczynnik uczenia alfa
  * współczynnik losowości eps
  * współczynnik propagacji błędów gamma